In [ ]:
import math
import numpy as np
import random, sys
import matplotlib.pyplot as plt
import _pickle as cPickle

In [ ]:
# Loading the dataset
dataset_full = cPickle.load(open("RML2016.10a_dict.dat",'rb'), encoding='latin1')

#snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], dataset_full.keys())))), [1,0])
#print(mods)



mods = ['BPSK', 'QAM16', 'AM-DSB', 'WBFM']#, 'AM-SSB', 'QPSK']

## Filter to consider only 3 modulations (BPSK, QPSK, QAM16) with SNR ranging from 0-18.
Xd = {}

for mod in mods:
    for snr in range(8,20,2):
        Xd[(mod,snr)] = dataset_full[(mod,snr)]

snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
print(mods)
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)
lbl = np.array(lbl)
print(lbl)
print(X.shape)
np.savetxt("labels.csv", lbl, delimiter=",", fmt="%s")


In [ ]:
## Create a Spectrogram and store it as a numpy array

from scipy import signal
import matplotlib.pyplot as plt
from PIL import Image


dataset = []
filename = "spectrograms.csv"
f_handle = open(filename, 'ab')


for i in range(0,len(X)):
    
    ## Convert the data into complex form
    dat = X[i,0,:] + 1j*X[i,1,:]  

    # Plot the spctrogram    
    plt.figure()
    ## reset the figure size
    fig_size = plt.rcParams["figure.figsize"]

    # Set figure width to 2.5 and height to 3
    fig_size[0] = 3
    fig_size[1] = 2.5
    
    plt.specgram(dat, NFFT=1024, Fs=1000000)
    #plt.title("PSD of 'signal' " + mod )
    #plt.xlabel("Time")
    #plt.ylabel("Frequency")
    axes = plt.gca()
    plt.axis('off')
    #axes.set_xlim([xmin,xmax])
    axes.set_ylim([-200000,200000])

    plt.rcParams["figure.figsize"] = fig_size


    ## Save the color image and reread it as a grayscale image
    plt.savefig("spect.png", bbox_inches='tight',pad_inches=-0.8)
    img = Image.open('spect.png').convert('L')


    ## Store the image as a numpy array
    img_as_np = np.asarray(img)
    #print(img_as_np.shape)
    #img.save('spect_gray.png')
    plt.close('all')

    dataset.append(img_as_np.flatten())
    
    
    ## Write the dataset into a csv in batches of 5000
    if(i%5000 == 0):
        dataset = np.array(dataset)
        np.savetxt(f_handle, dataset, delimiter=",", fmt="%3u")
        dataset = []
        print(i)

#print(dataset)
if dataset:
    dataset = np.array(dataset)
    np.savetxt(f_handle, dataset, delimiter=",", fmt="%3u")
        
    #np.save(f_handle, dataset)
    dataset = []
    
f_handle.close()            


print("Dataset and labels sucessfully generated and dumped into 'spectrograms.csv' and labels.csv files!")


In [ ]:
import pandas as pd
from sklearn.utils import shuffle
#from sklearn.cross_validation import train_test_split
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn import preprocessing



img_rows, img_cols = 41, 108
nb_classes = 4

## Change the modulation labels to numeric labels
le = preprocessing.LabelEncoder()
label_encoding = le.fit(lbl)
labels_numeric = le.transform(lbl) 

print(lbl)
print(labels_numeric)
print(label_encoding)
    
## Shuffle the data before test-train split    
data, Label = shuffle(dataset, labels_numeric, random_state = 2)
train_data = [data,Label]
    
(X,y) = (train_data[0], train_data[1])
    
## Split data into training and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
X_train = X_train.reshape(X_train.shape[0], img_cols, img_rows, 1)
X_test = X_test.reshape(X_test.shape[0], img_cols, img_rows, 1)    

#X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#X_test = X_test.reshape(X_test.shape[0],1,img_rows, img_cols)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
    
X_train /= 255
X_test /= 255
    
#print(X_train)
    
print('X_train shape: ', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
     
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

i=100
plt.imshow(X_train[i,0], interpolation = 'nearest')
plt.show()
print("label: ", Y_train[i,:])


In [ ]:
## Build the CNN Model

from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import SGD, RMSprop, adam
from keras.utils import np_utils

## Set Model Parameters

# batch size to train
batch_size = 32

# number of epochs to train
nb_epoch = 20

# number of channels
img_channels = 1

# number of convolutional filters to use
nb_filters = 32

# Size of pooling area for max pooling
nb_pool = 2

# convolution kernal size
nb_conv = 3

print("test")

model = Sequential()
print("abc")

model.add(Convolution2D(nb_filters, (nb_conv, nb_conv), padding='valid', input_shape=(img_cols, img_rows,1)))#,img_rows, img_cols)))
print("0")

model.add(Activation('relu'))
print('1')

model.add(Convolution2D(nb_filters, (nb_conv, nb_conv)))
model.add(Activation('relu'))
print("2")

model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.5))
print(3)

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

print(5)

#model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(X_test, Y_test) )

model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=0.2)


In [ ]:
## Store the test and train data

with open('train_x.dat','wb') as train_handle:
    np.save(train_handle, X_train)

with open('train_y.dat','wb') as train_lbl_handle:
    np.save(train_lbl_handle, Y_train)
    
with open('test_x.dat','wb') as test_handle:    
    np.save(test_handle, X_test)

with open('test_y.dat','wb') as test_lbl_handle:
    np.save(test_lbl_handle, Y_test)

train_handle.close()
test_handle.close()
train_lbl_handle.close()
test_lbl_handle.close()

## Store the model
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [ ]:
## Score the model
score = model.evaluate(X_test, Y_test, verbose=0)
print("Test Score: ", score[0])
print("Test Accuracy: ", score[1])

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([nb_classes,nb_classes])
confnorm = np.zeros([nb_classes,nb_classes])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,nb_classes):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=mods)